In [16]:
# Import Modules
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
#import yfinance as yf
from pathlib import Path
import sqlalchemy as sql
from MCForecastTools import MCSimulation

%matplotlib inline

In [17]:
load_dotenv("./SAMPLE.env")

True

In [18]:
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [19]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [20]:
tickers = ["RYT","XSW","XLK","USRT","XLRE","RWR","BBUS","JMOM","SPMD","MDYG","SLY","SLYG","SPY", "QQQ", "GLD"]
timeframe = "1D"
start_date = pd.Timestamp("2019-10-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-10-01", tz="America/New_York").isoformat()

In [21]:
df_hist_data = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date,
    limit = 1000
).df

In [22]:
df_hist_data.head()

BBUS                                         GLD  \
                            open     high      low    close   volume    open   
time                                                                           
2019-10-01 00:00:00-04:00  53.63  53.6400  52.8100  52.8100   9427.0  138.07   
2019-10-02 00:00:00-04:00  52.53  52.5300  51.7468  51.8799  16333.0  140.79   
2019-10-03 00:00:00-04:00  51.89  52.1600  51.3300  52.1600  17298.0  141.36   
2019-10-04 00:00:00-04:00  52.50  52.7701  52.4900  52.7701  10647.0  141.48   
2019-10-07 00:00:00-04:00  52.87  53.1400  52.8200  52.8200  11417.0  141.16   

                                                                ...   XLRE  \
                               high      low   close    volume  ...   open   
time                                                            ...          
2019-10-01 00:00:00-04:00  140.1900  137.800  139.62  10224515  ...  39.30   
2019-10-02 00:00:00-04:00  141.8800  140.462  141.27  10128824  ...  38.95   
2019-10-03 00:00:00-04:00  143.2500  141.290  141.91  10327887  ...  38.76   
2019-10-04 00:00:00-04:00  142.3100  141.140  141.93   8152981  ...  39.28   
2019-10-07 00:00:00-04:00  141.7709  140.240  140.72   5391425  ...  39.30   

                                                               XSW           \
                              high      low  close   volume   open     high   
time                                                                          
2019-10-01 00:00:00-04:00  39.3650  38.8500  38.97  5080856  93.59  93.9836   
2019-10-02 00:00:00-04:00  39.0450  38.5300  38.75  4891211  91.72  91.7200   
2019-10-03 00:00:00-04:00  39.3000  38.7000  39.22  5886450  91.00  92.5346   
2019-10-04 00:00:00-04:00  39.4898  39.2400  39.43  2482671  92.88  93.9600   
2019-10-07 00:00:00-04:00  39.5100  39.1499  39.32  2958281  93.70  94.5381   

                                                    
                               low  close   volume  
time                                                
2019-10-01 00:00:00-04:00  92.2100  92.32  30947.0  
2019-10-02 00:00:00-04:00  90.3001  91.08  24876.0  
2019-10-03 00:00:00-04:00  89.9100  92.41  39520.0  
2019-10-04 00:00:00-04:00  92.7883  93.95  45231.0  
2019-10-07 00:00:00-04:00  93.7000  94.26  13309.0  

[5 rows x 75 columns]

In [23]:
df_hist_data.tail()

BBUS                                     GLD  \
                            open    high     low  close   volume    open   
time                                                                       
2021-09-27 00:00:00-04:00  81.32  81.520  81.121  81.27  36219.0  163.88   
2021-09-28 00:00:00-04:00  80.67  80.690  79.480  79.56  27047.0  162.73   
2021-09-29 00:00:00-04:00  79.79  80.120  79.590  79.62  32278.0  162.35   
2021-09-30 00:00:00-04:00  79.94  80.030  78.720  78.72  25225.0  162.66   
2021-10-01 00:00:00-04:00  79.09  79.935  78.490  79.56  39710.0  164.27   

                                                              ...   XLRE  \
                              high      low   close   volume  ...   open   
time                                                          ...          
2021-09-27 00:00:00-04:00  164.365  163.510  163.65  4493388  ...  45.92   
2021-09-28 00:00:00-04:00  162.820  161.730  162.06  8410601  ...  44.87   
2021-09-29 00:00:00-04:00  162.740  160.970  161.36  7813632  ...  45.06   
2021-09-30 00:00:00-04:00  164.950  162.340  164.19  9592918  ...  45.38   
2021-10-01 00:00:00-04:00  164.960  163.885  164.58  6087266  ...  44.79   

                                                               XSW          \
                             high     low  close    volume    open    high   
time                                                                         
2021-09-27 00:00:00-04:00  46.140  45.105  45.13   6621245  178.39  178.94   
2021-09-28 00:00:00-04:00  45.080  44.530  44.87  11755550  175.80  175.80   
2021-09-29 00:00:00-04:00  45.380  45.000  45.18   5669530  172.98  173.47   
2021-09-30 00:00:00-04:00  45.395  44.450  44.47   5820243  172.37  173.27   
2021-10-01 00:00:00-04:00  45.250  44.400  44.93   8033142  172.65  174.93   

                                                       
                                low   close    volume  
time                                                   
2021-09-27 00:00:00-04:00  176.6798  177.98  193398.0  
2021-09-28 00:00:00-04:00  172.1200  172.12    9133.0  
2021-09-29 00:00:00-04:00  171.7800  171.80    8595.0  
2021-09-30 00:00:00-04:00  171.4448  171.63   13834.0  
2021-10-01 00:00:00-04:00  171.3850  174.62    6558.0  

[5 rows x 75 columns]

In [32]:
def performance_forecast(etf_df, sim_num=500, term=10):
    
    # Compute MC Confidence Interval for 'term' years
    MC_sim = MCSimulation(
        portfolio_data = etf_df,
        num_simulation = sim_num,
        num_trading_days = 252*term)
    cum_return = MC_sim.summarize_cumulative_return()
    mean = cum_return[1]
    std = cum_return[2]
    median = cum_return[5]
    ci_lower = cum_return[8]
    ci_upper = cum_return[9]
    
    return ci_lower, ci_upper, mean, std, median 

In [33]:
# Load etf list from csv
csv_path = Path('./Resources/etf_list.csv')
etf_list = pd.read_csv(csv_path)['etf']

etf_list_3extra = ["SPY", "QQQ", "GLD"]

# Initialize parameters
etf_final_list = []
ci_lower_10yrs_list = []
ci_upper_10yrs_list = []
mean_10yrs_list = []
std_10yrs_list = []
median_10yrs_list =[]
ci_lower_20yrs_list = []
ci_upper_20yrs_list = []
mean_20yrs_list = []
std_20yrs_list = []
median_20yrs_list =[]
ci_lower_30yrs_list = []
ci_upper_30yrs_list = []
mean_30yrs_list = []
std_30yrs_list = []
median_30yrs_list =[]

# Loop over etf list and compute Confidence Interval for each etf
for i in range(len(etf_list)):
    current_etf = etf_list[i]
    print("Working on ETF: " + current_etf)
    etf_df = df_hist_data.loc[:,[current_etf]]
    # Do 10, 20, 30 years MC for current etf respectively
    print("Computing 10 years return ... ")
    ci_lower_10yrs, ci_upper_10yrs, mean_10yrs, std_10yrs, median_10yrs = performance_forecast(etf_df, sim_num=500, term=10)
    print("Computing 20 years return ... ")
    ci_lower_20yrs, ci_upper_20yrs, mean_20yrs, std_20yrs, median_20yrs = performance_forecast(etf_df, sim_num=500, term=20)
    print("Computing 30 years return ... ")
    ci_lower_30yrs, ci_upper_30yrs, mean_30yrs, std_30yrs, median_30yrs = performance_forecast(etf_df, sim_num=500, term=30)
    
    # Store results into list
    etf_final_list.append(current_etf)
    ci_lower_10yrs_list.append(ci_lower_10yrs)
    ci_upper_10yrs_list.append(ci_upper_10yrs)
    mean_10yrs_list.append(mean_10yrs)
    std_10yrs_list.append(std_10yrs)
    median_10yrs_list.append(median_10yrs)
    ci_lower_20yrs_list.append(ci_lower_20yrs)
    ci_upper_20yrs_list.append(ci_upper_20yrs)
    mean_20yrs_list.append(mean_20yrs)
    std_20yrs_list.append(std_20yrs)
    median_20yrs_list.append(median_20yrs)
    ci_lower_30yrs_list.append(ci_lower_30yrs)
    ci_upper_30yrs_list.append(ci_upper_30yrs)
    mean_30yrs_list.append(mean_30yrs)
    std_30yrs_list.append(std_30yrs)
    median_30yrs_list.append(median_30yrs)
    
# Loop over 3 extra etf and compute Confidence Interval for each etf
for i in range(len(etf_list_3extra)):
    current_etf = etf_list_3extra[i]
    print("Working on ETF: " + current_etf)
    etf_df = df_hist_data.loc[:,[current_etf]]
    # Do 10, 20, 30 years MC for current etf respectively
    print("Computing 10 years return ... ")
    ci_lower_10yrs, ci_upper_10yrs, mean_10yrs, std_10yrs, median_10yrs = performance_forecast(etf_df, sim_num=500, term=10)
    print("Computing 20 years return ... ")
    ci_lower_20yrs, ci_upper_20yrs, mean_20yrs, std_20yrs, median_20yrs = performance_forecast(etf_df, sim_num=500, term=20)
    print("Computing 30 years return ... ")
    ci_lower_30yrs, ci_upper_30yrs, mean_30yrs, std_30yrs, median_30yrs = performance_forecast(etf_df, sim_num=500, term=30)
    
    # Store results into list
    etf_final_list.append(current_etf)
    ci_lower_10yrs_list.append(ci_lower_10yrs)
    ci_upper_10yrs_list.append(ci_upper_10yrs)
    mean_10yrs_list.append(mean_10yrs)
    std_10yrs_list.append(std_10yrs)
    median_10yrs_list.append(median_10yrs)
    ci_lower_20yrs_list.append(ci_lower_20yrs)
    ci_upper_20yrs_list.append(ci_upper_20yrs)
    mean_20yrs_list.append(mean_20yrs)
    std_20yrs_list.append(std_20yrs)
    median_20yrs_list.append(median_20yrs)
    ci_lower_30yrs_list.append(ci_lower_30yrs)
    ci_upper_30yrs_list.append(ci_upper_30yrs)
    mean_30yrs_list.append(mean_30yrs)
    std_30yrs_list.append(std_30yrs)
    median_30yrs_list.append(median_30yrs)

# Create ETF performance datafram
etf_performance_df = pd.DataFrame()
etf_performance_df['ETF'] = etf_final_list
etf_performance_df['ci_lower_10yrs'] = ci_lower_10yrs_list
etf_performance_df['ci_upper_10yrs'] = ci_upper_10yrs_list
etf_performance_df['mean_10yrs'] = mean_10yrs_list
etf_performance_df['std_10yrs'] = std_10yrs_list
etf_performance_df['median_10yrs'] = median_10yrs_list
etf_performance_df['ci_lower_20yrs'] = ci_lower_20yrs_list
etf_performance_df['ci_upper_20yrs'] = ci_upper_20yrs_list
etf_performance_df['mean_20yrs'] = mean_20yrs_list
etf_performance_df['std_20yrs'] = std_20yrs_list
etf_performance_df['median_20yrs'] = median_20yrs_list
etf_performance_df['ci_lower_30yrs'] = ci_lower_30yrs_list
etf_performance_df['ci_upper_30yrs'] = ci_upper_30yrs_list
etf_performance_df['mean_30yrs'] = mean_30yrs_list
etf_performance_df['std_30yrs'] = std_30yrs_list
etf_performance_df['median_30yrs'] = median_30yrs_list

etf_performance_df

Working on ETF: RYT
Computing 10 years return ... 
Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/Users/mingluli/Documents/cu-virt-fin-pt-09-2021-u-b-main/Project_Dashboard/MCForecastTools.py:127: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,ETF,ci_lower_10yrs,ci_upper_10yrs,mean_10yrs,std_10yrs,median_10yrs,ci_lower_20yrs,ci_upper_20yrs,mean_20yrs,std_20yrs,median_20yrs,ci_lower_30yrs,ci_upper_30yrs,mean_30yrs,std_30yrs,median_30yrs
0,RYT,1.844844,81.376439,18.825244,21.638775,11.997545,10.094816,1672.709842,353.227271,561.779986,164.361002,77.016361,63185.811822,7528.326817,20277.055978,1984.035631
1,XSW,4.014547,183.296011,39.438647,50.633110,23.645799,52.909431,5981.174683,1153.669340,1746.000844,570.348660,614.744852,294768.381613,50717.774685,132560.779110,15629.741824
2,XLK,3.728779,171.845210,42.098172,48.583371,26.271747,43.010426,11073.609877,1679.775439,3457.237748,639.654363,591.284742,354005.715120,54952.846305,169286.504046,12667.068599
3,USRT,0.215532,13.542962,2.852668,4.049903,1.511093,0.118266,39.743224,6.070057,12.372842,2.346964,0.087632,144.514060,17.512698,54.859574,3.187231
4,XLRE,0.286819,12.104682,3.030889,3.291873,1.973933,0.334226,56.096938,10.633570,24.935080,4.206110,0.341095,255.765444,33.642431,83.706145,7.554871
5,RWR,0.128798,9.395493,2.079896,3.444170,1.127254,0.077231,32.961900,4.321838,9.237027,1.270136,0.032389,47.818560,9.398889,43.904291,1.641182
6,BBUS,1.800651,37.145858,11.617188,11.432052,8.149110,6.699238,510.618302,114.701697,183.941431,62.614185,36.753749,5683.028907,1004.291196,2054.589755,403.505545
7,JMOM,2.010309,37.971220,11.783916,9.853473,8.700442,9.703486,676.558443,147.131067,261.156907,70.818653,35.547095,15727.129010,2120.977277,5107.007195,698.410033
8,SPMD,0.852998,38.962869,9.600496,11.110573,5.891003,2.257828,389.996786,75.861232,144.509633,33.969756,5.900234,3981.551403,665.698912,1898.176049,170.141593
9,MDYG,0.911756,36.009955,9.600054,10.523072,6.570101,2.573720,689.282061,119.760130,292.913793,42.546139,15.977656,5028.571315,901.285708,2787.539174,236.719938


In [34]:
# Review ETF performance dataframe
etf_performance_df

,ETF,ci_lower_10yrs,ci_upper_10yrs,mean_10yrs,std_10yrs,median_10yrs,ci_lower_20yrs,ci_upper_20yrs,mean_20yrs,std_20yrs,median_20yrs,ci_lower_30yrs,ci_upper_30yrs,mean_30yrs,std_30yrs,median_30yrs
0,RYT,1.844844,81.376439,18.825244,21.638775,11.997545,10.094816,1672.709842,353.227271,561.779986,164.361002,77.016361,63185.811822,7528.326817,20277.055978,1984.035631
1,XSW,4.014547,183.296011,39.438647,50.633110,23.645799,52.909431,5981.174683,1153.669340,1746.000844,570.348660,614.744852,294768.381613,50717.774685,132560.779110,15629.741824
2,XLK,3.728779,171.845210,42.098172,48.583371,26.271747,43.010426,11073.609877,1679.775439,3457.237748,639.654363,591.284742,354005.715120,54952.846305,169286.504046,12667.068599
3,USRT,0.215532,13.542962,2.852668,4.049903,1.511093,0.118266,39.743224,6.070057,12.372842,2.346964,0.087632,144.514060,17.512698,54.859574,3.187231
4,XLRE,0.286819,12.104682,3.030889,3.291873,1.973933,0.334226,56.096938,10.633570,24.935080,4.206110,0.341095,255.765444,33.642431,83.706145,7.554871
5,RWR,0.128798,9.395493,2.079896,3.444170,1.127254,0.077231,32.961900,4.321838,9.237027,1.270136,0.032389,47.818560,9.398889,43.904291,1.641182
6,BBUS,1.800651,37.145858,11.617188,11.432052,8.149110,6.699238,510.618302,114.701697,183.941431,62.614185,36.753749,5683.028907,1004.291196,2054.589755,403.505545
7,JMOM,2.010309,37.971220,11.783916,9.853473,8.700442,9.703486,676.558443,147.131067,261.156907,70.818653,35.547095,15727.129010,2120.977277,5107.007195,698.410033
8,SPMD,0.852998,38.962869,9.600496,11.110573,5.891003,2.257828,389.996786,75.861232,144.509633,33.969756,5.900234,3981.551403,665.698912,1898.176049,170.141593
9,MDYG,0.911756,36.009955,9.600054,10.523072,6.570101,2.573720,689.282061,119.760130,292.913793,42.546139,15.977656,5028.571315,901.285708,2787.539174,236.719938
